In [36]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from datetime import datetime

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import sys
sys.path.append('./datasets')
sys.path.append('./models')

from UNSW_NB15 import UNSWNB15Dataset
from KDDCUP99 import KDDCUP99Dataset
from CICIDS2017 import CICIDS2017Dataset

from StandardAE import StandardAE

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median').set_output(transform='pandas')),
    ('scaler', StandardScaler().set_output(transform='pandas'))
])

In [7]:
data_dir = Path("../data/KDDCUP99/preprocessed")
KDDCUP99 = {
    'train' : KDDCUP99Dataset(data_dir, type = "train", transformer = pipeline),
    'val' : KDDCUP99Dataset(data_dir, type = "val", transformer = pipeline),
    'test' : KDDCUP99Dataset(data_dir, type = "test", transformer = pipeline)
}



Loaded train data with 912188 samples.
Data shape: (912188, 125)
num_anomalies: 0
num_normal: 912188
anomaly ratio: 0.0

Loaded val data with 311029 samples.
Data shape: (311029, 125)
num_anomalies: 250436
num_normal: 60593
anomaly ratio: 0.8051853685669182

Loaded test data with 311029 samples.
Data shape: (311029, 125)
num_anomalies: 250436
num_normal: 60593
anomaly ratio: 0.8051853685669182


In [9]:
data_dir = Path("../data/CIC-IDS2017/preprocessed")
CICIDS2017 = {
    'train' : CICIDS2017Dataset(data_dir, type = "train", transformer = pipeline),
    'val' : CICIDS2017Dataset(data_dir, type = "val", transformer = pipeline),
    'test' : CICIDS2017Dataset(data_dir, type = "test", transformer = pipeline)
}


Loaded train data with 1715451 samples.
Data shape: (1715451, 80)
num_anomalies: 0
num_normal: 1715451
anomaly ratio: 0.0
columns with missing values:
['Flow Bytes/s', 'Flow Packets/s']
columns with infinity values:
[]

Loaded val data with 557646 samples.
Data shape: (557646, 80)
num_anomalies: 278823
num_normal: 278823
anomaly ratio: 0.5
columns with missing values:
['Flow Bytes/s', 'Flow Packets/s']
columns with infinity values:
[]

Loaded test data with 557646 samples.
Data shape: (557646, 80)
num_anomalies: 278823
num_normal: 278823
anomaly ratio: 0.5
columns with missing values:
['Flow Bytes/s', 'Flow Packets/s']
columns with infinity values:
[]


In [8]:
data_dir = Path("../data/UNSW-NB15/preprocessed")
UNSWNB15 = {
    'train' : UNSWNB15Dataset(data_dir, type = "train", transformer = pipeline),
    'val' : UNSWNB15Dataset(data_dir, type = "val", transformer = pipeline),
    'test' : UNSWNB15Dataset(data_dir, type = "test", transformer = pipeline)
}


Loaded train data with 46000 samples.
Data shape: (46000, 196)
num_anomalies: 0
num_normal: 46000
anomaly ratio: 0.0

Loaded val data with 22252 samples.
Data shape: (22252, 196)
num_anomalies: 12252
num_normal: 10000
anomaly ratio: 0.5506021930612979

Loaded test data with 82332 samples.
Data shape: (82332, 196)
num_anomalies: 45332
num_normal: 37000
anomaly ratio: 0.5506000097167566


In [34]:
def run_experiment(model, 
                   dataset,
                   max_epochs=10,
                   experiment_name="undefined",
                   run_name="undefined",
                   dataset_name="undefined",
                   save_model=False):

    model.set_tech_params(
        accelerator='gpu',
        batch_size=1024, 
        num_workers=1, 
        persistent_workers=False
    )

    model.fit(dataset['train'], dataset['val'], max_epochs=max_epochs, log=True, 
                        logger_params = {
                                "experiment_name": experiment_name,
                                "run_name": run_name,
                                "log_model": False,
                                "tags": {"dataset": dataset_name},
                        })
    
    if save_model:
        save_dir = Path(f"../saved_models/{experiment_name}/{run_name}")
        file_name = f"{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_{model.__class__.__name__}"
        file_num = 0
        while (save_dir / f"{file_name}_{file_num}.pt").exists():
            file_num += 1

        model.save(save_dir / f"{file_name}_{file_num}.pt")

In [37]:
UNSWNB15_input_size = UNSWNB15['train'][0][0].shape[0]

IStandardAE_model = StandardAE(input_size=UNSWNB15_input_size, 
                                hidden_sizes=[128,32, 8], 
                                dropout=False, 
                                initial_lr=2e-3, 
                                linear_lr_start_factor=1, 
                                linear_lr_end_factor=0.03, 
                                linear_lr_total_iters=25)

run_experiment(model=IStandardAE_model,
                dataset=UNSWNB15,
                max_epochs=1,
                experiment_name="IADModel test",
                run_name="run_experiment test",
                dataset_name="UNSW-NB15",
                save_model=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Experiment with name IADModel test not found. Creating it.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 29.4 K | train
1 | decoder | Sequential | 29.5 K | train
-----------------------------------------------
58.9 K    Trainable params
0         Non-trainable params
58.9 K    Total params
0.236     Total estimated model params size (MB)
12        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [28]:
predictions = IStandardAE_model.predict_raw(UNSWNB15['val'])